# Начало работы

## Загрузка библиотек

In [1]:
import pandas as pd
import seaborn as sns

## загрузка датасета, базовый просмотр данных

In [2]:
try:
    df = pd.read_csv('games.csv')
    print('local load successful')
except:
    print('local load failed, loading from the inernet')
    df = pd.read_csv('https://code.s3.yandex.net/datasets/games.csv')
display(df.sample(10))
display(df.info())

local load successful


,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
9434,Wantame Music Channel: Doko Demo Style,DS,2007.0,Misc,0.00,0.00,0.13,0.00,NaN,NaN,NaN
5917,Hamster Tarou,GB,1999.0,Simulation,0.00,0.00,0.30,0.00,NaN,NaN,NaN
8640,Field Commander,PSP,2006.0,Strategy,0.14,0.00,0.00,0.01,77.0,7.8,T
6776,Tongari Boushi to Oshare na Mahou Tsukai,DS,2011.0,Action,0.00,0.00,0.24,0.00,NaN,NaN,NaN
13012,Disney's PK: Out of the Shadows,GC,2002.0,Platform,0.04,0.01,0.00,0.00,46.0,tbd,E
1748,Sega Rally Championship,SAT,1995.0,Racing,0.21,0.16,0.77,0.02,NaN,NaN,NaN
12321,Insecticide,DS,2008.0,Action,0.06,0.00,0.00,0.00,55.0,7,E10+
4552,Atelier Totori: The Adventurer of Arland,PS3,2010.0,Role-Playing,0.12,0.06,0.21,0.03,74.0,6.7,T
13504,Rapala Trophies,PSP,2006.0,Sports,0.04,0.00,0.00,0.00,NaN,tbd,E
11843,Big Mutha Truckers 2,XB,2005.0,Racing,0.05,0.02,0.00,0.00,50.0,3.4,M


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


None

## Первые наблюдения
в датасете данные по играм  
всего 16715 игр  
по 2 играм отсутствует название игры  
по нескольким играм отсутствует год релиза  
`user_score` тип строка, потому что в некоторых строках имеются значения `tbd`  
довольно много отсутствует значений оценок игроков и критиков

## План по исправлению / заполнению пропущенных данных
1) заменить заглавные буквы в названиях колонок на строчные
2) убрать записи без названия игры
3) попробовать заполнить `year of release` и привести к типу `int`
4) `user_score` убрать `tbd` и привести к `float32`
5) все `sales` и `score` привести к типу `float32`
6) исследовать `score` на предмет возможности заполнения отсутствующих значений

In [3]:
df.columns = df.columns.str.lower()
# df.reset_index(drop=True, inplace=True)
df.sample(10)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
16515,Routes PE,PS2,2007.0,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
4432,Naruto: Gekito Ninja Taisen! 3,GC,2004.0,Fighting,0.00,0.00,0.43,0.01,NaN,NaN,NaN
11234,Just Dance Wii U,WiiU,2014.0,Misc,0.00,0.00,0.09,0.00,NaN,NaN,NaN
9188,Pro Yaky? Spirits 2014,PS3,2014.0,Sports,0.00,0.00,0.14,0.00,NaN,NaN,NaN
15134,Total Immersion Racing,XB,2002.0,Racing,0.02,0.01,0.00,0.00,60.0,tbd,E
11365,Metal Slug Advance,GBA,2004.0,Shooter,0.06,0.02,0.00,0.00,79.0,8.3,T
5565,Just Dance 2016,XOne,2015.0,Misc,0.25,0.04,0.00,0.03,66.0,6.1,E10+
7040,What's Cooking? Jamie Oliver,DS,2008.0,Simulation,0.20,0.01,0.00,0.02,NaN,NaN,NaN
10264,Real World Golf,PS2,2005.0,Sports,0.05,0.04,0.00,0.01,73.0,6.3,E
8344,FaceBreaker,PS3,2008.0,Fighting,0.13,0.02,0.00,0.02,53.0,3.8,T


In [4]:
df = df[~df.name.isna()]

In [11]:
condition = (df.year_of_release.isna()) & (df.name.str.contains('\d{4}', regex=True))
df.loc[condition,'year_of_release'] = df.loc[condition,'name'].str.extract('(\d{4})').astype('float64')

In [12]:
df[df.year_of_release.isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
183,Madden NFL 2004,PS2,NaN,Sports,4.26,0.26,0.01,0.71,94.0,8.5,E
377,FIFA Soccer 2004,PS2,NaN,Sports,0.59,2.36,0.04,0.51,84.0,6.4,E
456,LEGO Batman: The Videogame,Wii,NaN,Action,1.80,0.97,0.00,0.29,74.0,7.9,E10+
475,wwe Smackdown vs. Raw 2006,PS2,NaN,Fighting,1.57,1.02,0.00,0.41,NaN,NaN,NaN
609,Space Invaders,2600,NaN,Shooter,2.36,0.14,0.00,0.03,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16373,PDC World Championship Darts 2008,PSP,NaN,Sports,0.01,0.00,0.00,0.00,43.0,tbd,E10+
16405,Freaky Flyers,GC,NaN,Racing,0.01,0.00,0.00,0.00,69.0,6.5,T
16448,Inversion,PC,NaN,Shooter,0.01,0.00,0.00,0.00,59.0,6.7,M
16458,Hakuouki: Shinsengumi Kitan,PS3,NaN,Adventure,0.01,0.00,0.00,0.00,NaN,NaN,NaN


In [9]:
df[condition]['name'].str.extract('(\d{4})').astype('int')

,0
183,2004
377,2004
475,2006
1650,2003
2572,2009
3486,2002
4775,2003
5156,2003
5336,2013
5655,2005


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.5+ MB
